In [ ]:
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import OneHotEncoder
from tqdm.notebook import tqdm

random.seed(0)
np.random.seed(0)


df_train = pd.read_excel("train.xlsx")
df_test = pd.read_excel("test.xlsx")

In [ ]:
df_train.tail(50)

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
26124,26124,20220319-16563-127169023,1,12000.0,12000,Отложенная электронная оплата: Банк Россия (ба...,2022-03-05 12:32:48,NaT,2022-03-19 15:00:00,1,2022-03-20 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,4
26125,26125,20230621-6634-205272699,1,52900.0,8700,Система быстрых платежей: Эквайринг ComfortBoo...,2023-06-16 13:42:26,NaT,2023-06-21 15:00:00,5,2023-06-26 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1
26126,26126,20230130-6634-171381174,1,49470.0,0,Отложенная электронная оплата: Банк Россия (ба...,2022-11-17 09:14:25,2022-11-23 09:36:04,2023-01-30 15:00:00,6,2023-02-05 12:00:00,Официальный сайт,Отмена,Номер «Стандарт»,2,1
26127,26127,20231226-6634-231973428,1,139700.0,0,Отложенная электронная оплата: Банк Россия (ба...,2023-10-21 20:58:44,2023-10-22 09:03:36,2023-12-26 15:00:00,3,2023-12-29 12:00:00,Официальный сайт,Неподтвержденные,Коттедж с 3 спальнями,8,1
26128,26128,20231021-7491-231277842,1,12000.0,12000,Банк. карта: Банк Россия (банк. карта),2023-10-18 11:26:59,NaT,2023-10-21 15:00:00,1,2023-10-22 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,3,3
26129,26129,20221208-7491-174276822,1,6800.0,6800,Отложенная электронная оплата: Банк Россия (ба...,2022-12-07 12:57:14,NaT,2022-12-08 15:00:00,1,2022-12-09 12:00:00,Бронирование из экстранета,Активный,Номер «Стандарт»,1,3
26130,26130,20220717-7491-148716704,1,13700.0,13700,Банк. карта: Банк Россия (банк. карта),2022-07-10 13:13:06,NaT,2022-07-17 15:00:00,1,2022-07-18 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,3
26131,26131,20221015-7491-166313708,1,13700.0,0,Отложенная электронная оплата: Банк Россия (ба...,2022-10-13 21:05:33,2022-10-14 09:08:36,2022-10-15 15:00:00,1,2022-10-16 12:00:00,Бронирование из экстранета,Неподтвержденные,Номер «Стандарт»,3,3
26132,26132,20220921-7492-157590565,1,31740.0,16820,Банк. карта: Банк Россия (банк. карта),2022-08-23 12:02:43,NaT,2022-09-21 15:00:00,2,2022-09-23 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,2
26133,26133,20230710-7492-206532545,2,29600.0,29600,Отложенная электронная оплата: Банк Россия (ба...,2023-06-22 14:43:51,NaT,2023-07-10 15:00:00,1,2023-07-11 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,4,2


In [ ]:
def prepare_df(df):
    df = df.copy()
    df = df.drop(["Unnamed: 0", "№ брони"], axis=1)
    df["Дата бронирования"] = pd.to_datetime(df["Дата бронирования"])
    df["Заезд"] = pd.to_datetime(df["Заезд"])
    df["Выезд"] = pd.to_datetime(df["Выезд"])
    return df


def create_date_features(df, prefix):
    df = df.copy()
    df[prefix + "_month"] = df[prefix].dt.month.astype("int8")
    df[prefix + "_day_of_month"] = df[prefix].dt.day.astype("int8")
    df[prefix + "_day_of_year"] = df[prefix].dt.dayofyear.astype("int16")
    df[prefix + "_week_of_month"] = (
        df[prefix].apply(lambda d: (d.day - 1) // 7 + 1)
    ).astype("int8")
    df[prefix + "_week_of_year"] = (df[prefix].dt.isocalendar().week).astype("int8")
    df[prefix + "_day_of_week"] = (df[prefix].dt.dayofweek + 1).astype("int8")
    df[prefix + "_year"] = df[prefix].dt.year.astype("int32")
    df[prefix + "_is_wknd"] = (df[prefix].dt.weekday // 4).astype("int8")
    df[prefix + "_season"] = np.where(df[prefix + "_month"].isin([12, 1, 2]), 0, 1)
    df[prefix + "_season"] = np.where(
        df[prefix + "_month"].isin([6, 7, 8]), 2, df[prefix + "_season"]
    )
    df[prefix + "_season"] = pd.Series(
        np.where(df[prefix + "_month"].isin([9, 10, 11]), 3, df[prefix + "_season"])
    ).astype("int8")
    return df


def create_diff_features(df, prefix1, prefix2):
    df = df.copy()
    df[prefix1 + "_" + prefix2 + "_diff_in_days"] = (df[prefix1] - df[prefix2]).dt.days
    df[prefix1 + "_" + prefix2 + "_diff_in_weeks"] = (
        df[prefix1 + "_" + prefix2 + "_diff_in_days"] / 7
    )
    df[prefix1 + "_" + prefix2 + "_diff_in_hours"] = (
        df[prefix1] - df[prefix2]
    ).dt.total_seconds() / 3600
    return df


def create_payment_method_features(df):
    df = df.copy()
    df["SberPay"] = df["Способ оплаты"].apply(lambda x: int("SberPay" in x))
    df["Yandex Pay"] = df["Способ оплаты"].apply(lambda x: int("Yandex Pay" in x))
    df["МИР"] = df["Способ оплаты"].apply(lambda x: int("МИР" in x))
    df["ComfortBooking"] = df["Способ оплаты"].apply(lambda x: int("ComfortBooking" in x))
    df["TravelLine Pro"] = df["Способ оплаты"].apply(lambda x: int("TravelLine Pro" in x))
    df["Банк Россия"] = df["Способ оплаты"].apply(lambda x: int("Банк Россия" in x))
    df["Внешняя система оплаты"] = df["Способ оплаты"].apply(
        lambda x: int("Внешняя система оплаты" in x)
    )
    df["Банковская карта"] = df["Способ оплаты"].apply(
        lambda x: int(
            "Банковская карта" in x
            or "Банк. карта".lower() in x.lower()
            or "банковской картой".lower() in x.lower()
        )
    )
    df["Оплата наличными"] = df["Способ оплаты"].apply(
        lambda x: int("Оплата наличными" in x)
    )
    df["С предоплатой"] = df["Способ оплаты"].apply(lambda x: int("С предоплатой" in x))
    df["СБП"] = df["Способ оплаты"].apply(lambda x: int("Система быстрых платежей" in x))

    df["Отложенная электронная оплата"] = df["Способ оплаты"].apply(
        lambda x: int("Отложенная электронная оплата" in x)
    )

    df["Гарантия банковской картой"] = df["Способ оплаты"].apply(
        lambda x: int("Гарантия банковской картой" in x)
    )

    df["При заселении"] = df["Способ оплаты"].apply(lambda x: int("При заселении" in x))

    return df


def clear_source_column(text):
    # good
    mapping = [
        "Официальный сайт",
        "Бронирование из экстранета",
        "Яндекс.Путешествия",
        "ostrovok",
        "booking",
        "Программа лояльности",
        "Bronevik",
        "OneTwoTrip",
    ]

    for map_ in mapping:
        if map_.lower() in text.lower():
            return map_
    return "other"


def clear_category(text):
    # good
    if "\n" in text:
        text = text.split("\n")
        text = text[0]
    text = text.strip("1. ")
    return text

In [ ]:
def preprocess_df(df):
    df = df.copy()
    df = prepare_df(df)
    df = create_date_features(df, "Дата бронирования")
    df = create_date_features(df, "Заезд")
    df = create_date_features(df, "Выезд")
    df = create_diff_features(df, "Заезд", "Дата бронирования")
    df = create_diff_features(df, "Выезд", "Заезд")

    df = create_payment_method_features(df)

    # df["Источник"] = df["Источник"].apply(clear_source_column)

    df["Категория multiple selection"] = df["Категория номера"].apply(
        lambda x: int("\n" in x)
    )

    # df["Категория номера"] = df["Категория номера"].apply(clear_category)

    # other features

    df["Стоимость за ночь"] = df["Стоимость"] / df["Ночей"]
    df["Внесена предоплата binary"] = df.apply(
        lambda x: int(x["Внесена предоплата"] != 0), axis=1
    )

    df["Предоплата умноженная на время до прибытия"] = (
        df["Внесена предоплата"] * df["Заезд_Дата бронирования_diff_in_days"]
    )

    df["Процент предоплата от стоимости"] = df["Внесена предоплата"] / df["Стоимость"]
    df["Гостей на номер"] = df["Гостей"] / df["Номеров"]

    df["Стоимость на гостя"] = df["Стоимость"] / df["Гостей"]
    df["Стоимость за номер"] = df["Стоимость"] / df["Номеров"]
    df["Стоимость за ночь 1 номер"] = df["Стоимость"] / df["Ночей"] / df["Номеров"]
    df["Стоимость за ночь 1 гостя"] = df["Стоимость"] / df["Ночей"] / df["Гостей"]
    df["Гостей на номер"] = df["Гостей"] / df["Номеров"]
    df["Бронирование утром"] = df["Дата бронирования"].apply(
        lambda x: (
            1
            if x.time() >= pd.Timestamp("05:00:00").time()
            and x.time() < pd.Timestamp("11:00:00").time()
            else 0
        )
    )
    df["Бронирование вечером"] = df["Дата бронирования"].apply(
        lambda x: (
            1
            if x.time() >= pd.Timestamp("17:00:00").time()
            and x.time() < pd.Timestamp("23:00:00").time()
            else 0
        )
    )
    df["Бронирование ночью"] = df["Дата бронирования"].apply(
        lambda x: (
            1
            if x.time() >= pd.Timestamp("23:00:00").time()
            or x.time() < pd.Timestamp("05:00:00").time()
            else 0
        )
    )
    return df

In [ ]:
df_train = preprocess_df(df_train)
df_test = preprocess_df(df_test)

In [ ]:
# drops
df_train = df_train.drop(["Дата бронирования", "Заезд", "Выезд"], axis=1)
df_test = df_test.drop(["Дата бронирования", "Заезд", "Выезд"], axis=1)

In [ ]:
df_train["target"] = df_train["Дата отмены"].apply(lambda x: int(pd.notna(x)))
df_train = df_train.drop(["Дата отмены", "Статус брони"], axis=1)

In [ ]:
X_train = df_train.drop("target", axis=1)
y_train = df_train["target"]

In [ ]:
models_list = []
scores_list = []
y_pred = np.zeros(df_test.shape[0])
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
for i, (train_index, test_index) in enumerate(splitter.split(X_train, y_train)):
    X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
    X_fold_test, y_fold_test = X_train.iloc[test_index], y_train.iloc[test_index]

    model = CatBoostClassifier(
        cat_features=["Способ оплаты", "Источник", "Категория номера"],
        verbose=0,
        eval_metric="AUC",
        early_stopping_rounds=50,
    )

    model.fit(X_fold_train, y_fold_train, eval_set=(X_fold_test, y_fold_test))

    preds = model.predict_proba(X_fold_test)[:, 1]
    score = roc_auc_score(y_fold_test, preds)

    models_list.append(model)
    scores_list.append(score)


np.mean(scores_list), np.std(scores_list)

(0.8629498742553416, 0.01466667522472403)

In [ ]:
pd.DataFrame(zip(X_train, models_list[-1].feature_importances_)).sort_values(
    1, ascending=False
)

,0,1
60,Процент предоплата от стоимости,26.585192
58,Внесена предоплата binary,12.558086
47,Банк Россия,9.536047
53,Отложенная электронная оплата,8.233879
3,Способ оплаты,6.448990
...,...,...
43,Yandex Pay,0.000032
52,СБП,0.000000
42,SberPay,0.000000
16,Дата бронирования_is_wknd,0.000000


In [ ]:
y_pred = np.zeros(df_test.shape[0])

for model, score in zip(models_list, scores_list):
    y_pred += model.predict_proba(df_test)[:, 1]
y_pred = y_pred / len(models_list)

In [ ]:
X_train = pd.concat([X_train, df_test])
y_train = pd.concat(
    [y_train, pd.DataFrame((y_pred > 0.5).astype(int), columns=["target"])]
)

In [ ]:
models_list = []
scores_list = []
y_pred = np.zeros(df_test.shape[0])
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
for i, (train_index, test_index) in enumerate(splitter.split(X_train, y_train)):
    X_fold_train, y_fold_train = X_train.iloc[train_index], y_train.iloc[train_index]
    X_fold_test, y_fold_test = X_train.iloc[test_index], y_train.iloc[test_index]

    model = CatBoostClassifier(
        cat_features=["Способ оплаты", "Источник", "Категория номера"],
        verbose=0,
        eval_metric="AUC",
        early_stopping_rounds=50,
    )

    model.fit(X_fold_train, y_fold_train, eval_set=(X_fold_test, y_fold_test))

    preds = model.predict_proba(X_fold_test)[:, 1]
    score = roc_auc_score(y_fold_test, preds)

    models_list.append(model)
    scores_list.append(score)


np.mean(scores_list), np.std(scores_list)

(0.884942928643102, 0.008971985034050884)

In [ ]:
y_pred = np.zeros(df_test.shape[0])

for model, score in zip(models_list, scores_list):
    y_pred += model.predict_proba(df_test)[:, 1]
y_pred = y_pred / len(models_list)

In [ ]:
submissiom = pd.DataFrame(y_pred)
submissiom.to_csv(
    "catboost_trained_on_test_no_feature_cleaning.csv", index=False, header=False
)